In [2]:
import cv2
import numpy as np

## Q1 : Dans quels cas la transformation entre deux images est-elle une homographie?

Cette question ne nécessite pas de code, mais voici la réponse :

1. Lorsque les images sont des projections planes d'une même scène 3D.
2. Quand on change de point de vue sur un plan dans la scène.
3. Pour des rotations pures de la caméra (sans translation).
4. Lors de changements de focale de la caméra.

## Q2 : À partir du code fourni, essayez d'estimer l'homographie permettant de transformer l'image de la mosaïque Pompei.jpg en vue verticale (en supposant le cadre carré, voir Figure 1).

Le code pour cette partie est déjà fourni dans le script initial. Voici les parties importantes :

In [ ]:
import numpy as np
import cv2

print("Version d'OpenCV: ", cv2.__version__)

# Charger l'image
PATH_IMG = './Images_Homographie/'
img = cv2.imread(PATH_IMG + "Pompei.jpg")

if img is None:
    print("Erreur : impossible de charger l'image.")
    exit(1)

(h, w, c) = img.shape
print("Dimension de l'image :", h, "lignes x", w, "colonnes x", c, "couleurs")

# Variables globales pour le clic
points_selected = 0
X_init = []
clone = img.copy()

# Fonction de sélection des points
def select_points(event, x, y, flags, param):
    global points_selected, X_init, img, clone
    if event == cv2.EVENT_LBUTTONDOWN:  # Clic gauche pour sélectionner un point
        points_selected += 1
        cv2.circle(img, (x, y), 8, (0, 255, 255), -1)  # Dessiner un cercle
        cv2.line(img, (x - 8, y), (x + 8, y), (0, 255, 0), 1)  # Ligne horizontale
        cv2.line(img, (x, y - 8), (x, y + 8), (0, 255, 0), 1)  # Ligne verticale
        X_init.append([x, y])  # Ajouter le point sélectionné
    elif event == cv2.EVENT_RBUTTONDOWN:  # Clic droit pour réinitialiser
        points_selected = 0
        X_init = []
        img = clone.copy()

# Initialiser la fenêtre et le callback
cv2.namedWindow("Image initiale")
cv2.setMouseCallback("Image initiale", select_points)

# Afficher l'image et sélectionner les points
while True:
    cv2.imshow("Image initiale", img)
    key = cv2.waitKey(1) & 0xFF
    if (key == ord("q")) and (points_selected >= 4):  # Touche 'q' pour valider
        break

# Conversion des points initiaux en array numpy
X_init = np.asarray(X_init, dtype=np.float32)
print("Points initiaux (X_init) :", X_init)

# Demander les coordonnées des points correspondants
X_final = np.zeros((points_selected, 2), np.float32)
print("\nEntrez les coordonnées cibles des points correspondants (par exemple pour un carré parfait) :")
for i in range(points_selected):
    while True:
        try:
            coords = input(f"Correspondant pour {X_init[i]} (format: x y) : ")
            x, y = map(float, coords.split())
            X_final[i] = [x, y]
            break
        except ValueError:
            print("Entrée invalide. Veuillez entrer deux nombres séparés par un espace.")

print("Points finaux (X_final) :", X_final)

# Calcul de l'homographie
H, status = cv2.findHomography(X_init, X_final, cv2.RANSAC)
if H is None:
    print("Erreur : le calcul de l'homographie a échoué.")
    exit(1)

# Application de l'homographie
img_warp = cv2.warpPerspective(clone, H, (w, h))
cv2.imshow("Image rectifiée", img_warp)

# Sauvegarde ou sortie
while True:
    key = cv2.waitKey(0)
    if key == ord("q"):  # Quitter sans sauvegarder
        cv2.destroyAllWindows()
        break
    elif key == ord("s"):  # Sauvegarder l'image
        cv2.imwrite("img_rectified.png", img_warp)
        print("Image sauvegardée sous 'img_rectified.png'.")
        cv2.destroyAllWindows()
        break


Version d'OpenCV:  4.10.0
Dimension de l'image : 333 lignes x 500 colonnes x 3 couleurs


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/youssef/Desktop/Labs_Computer_Vision/new-venv/lib/python3.12/site-packages/cv2/qt/plugins"


### Combien de points au minimum devez-vous sélectionner ?

Pour estimer une homographie, **au moins 4 points correspondants** sont nécessaires. 

#### **Explication :**
1. **Homographie :**  
   Une homographie \( H \) est une transformation projective définie par une matrice \( 3 \times 3 \) :
   $$
   H = \begin{pmatrix} 
   h_{11} & h_{12} & h_{13} \\
   h_{21} & h_{22} & h_{23} \\
   h_{31} & h_{32} & 1
   \end{pmatrix} 
   $$
   Cette matrice a **8 degrés de liberté** (9 paramètres, mais l'échelle est arbitraire, donc \( h_{33} = 1 \)).

2. **Équations par point :**  
   Chaque point de correspondance \((x, y) \leftrightarrow (x', y')\) génère deux équations linéaires :
   $$
   x' = \frac{h_{11}x + h_{12}y + h_{13}}{h_{31}x + h_{32}y + 1}, \quad
   y' = \frac{h_{21}x + h_{22}y + h_{23}}{h_{31}x + h_{32}y + 1}.
   $$
   Ces équations peuvent être réécrites sous forme matricielle.

3. **Nombre d'équations nécessaires :**  
   Chaque point de correspondance fournit 2 équations. Ainsi, pour résoudre un système d'équations avec 8 inconnues (les coefficients de \( H \)), **4 points sont nécessaires** :
   $$
   4 \text{ points } \times 2 \text{ équations par point } = 8 \text{ équations.}
   $$

- danc Le **minimum requis** est **4 points correspondants** pour calculer l'homographie.  

### Q3 : Pour estimer l'homographie, créez la matrice 2n×9 A comme indiqué en cours à partir de vos n correspondances, puis résolvez le système Ah=0 en utilisant la décomposition en valeurs singulières de numpy.

Voici le code pour estimer l'homographie :

In [ ]:
import cv2
import numpy as np

# Fonction pour estimer l'homographie
def estimate_homography(points1, points2):
    # Créer la matrice A de dimension 2n x 9
    A = []
    for i in range(len(points1)):
        x1, y1 = points1[i]
        x2, y2 = points2[i]
        A.append([-x1, -y1, -1, 0, 0, 0, x2 * x1, x2 * y1, x2])
        A.append([0, 0, 0, -x1, -y1, -1, y2 * x1, y2 * y1, y2])
    
    A = np.array(A)

    # Appliquer la décomposition en valeurs singulières
    U, S, V = np.linalg.svd(A)
    
    # Extraire le dernier vecteur de V (associé à la plus petite valeur singulière)
    h = V[-1]
    
    # Reshape h pour obtenir la matrice H 3x3
    H = h.reshape(3, 3)
    
    return H

# Fonction pour appliquer l'homographie sur l'image
def apply_homography(img1, img2, H):
    # Appliquer la transformation sur l'image
    height, width = img1.shape[:2]
    img2_warped = cv2.warpPerspective(img2, H, (width, height))
    return img2_warped

# Exemple d'utilisation avec des points correspondants
points1 = [(100, 150), (200, 250), (300, 350), (400, 450)]  # Points dans la première image
points2 = [(110, 160), (210, 260), (310, 360), (410, 460)]  # Points dans la deuxième image

# Estimer l'homographie
H = estimate_homography(points1, points2)

# Charger les images
img1 = cv2.imread('./Images_Homographie/paris_a.jpg')  # Remplacer par votre image
img2 = cv2.imread('./Images_Homographie/paris_b.jpg')  # Remplacer par votre image

# Appliquer l'homographie pour transformer img2 dans le référentiel de img1
img2_warped = apply_homography(img1, img2, H)

# Afficher l'image résultante
cv2.imshow("Warped Image", img2_warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Afficher la matrice d'homographie
print("Matrice d'homographie H :")
print(H)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/youssef/Desktop/Labs_Computer_Vision/new-venv/lib/python3.12/site-packages/cv2/qt/plugins"


# L’intérêt du traitement de l’homographie

Le traitement de l’homographie a plusieurs intérêts clés, principalement dans le domaine de la vision par ordinateur et de la géométrie projective. Voici quelques-uns des principaux objectifs :

1. **Alignement d'images prises sous des angles différents** :  
   L'homographie permet d’ajuster ou de "transformer" une image pour qu’elle corresponde à une autre, même si elles ont été prises sous des perspectives différentes. Cela est essentiel pour des applications telles que la création de panoramas, où plusieurs images doivent être fusionnées de manière transparente pour former une seule vue d'ensemble.

2. **Correction de la perspective** :  
   L'homographie permet de corriger les déformations dues à la perspective dans les images. Par exemple, dans les photographies d’un objet pris sous différents angles, l’homographie permet de "redresser" l’image en fonction d’un plan de référence.

3. **Modélisation de scènes 3D et réalité augmentée** :  
   Dans des systèmes de réalité augmentée ou de modélisation 3D, l’homographie est utilisée pour estimer la transformation entre les vues d’une scène ou d’un objet. Cela aide à superposer des objets virtuels sur des vues réelles, ajustant correctement la position et la perspective.

4. **Suivi d'objets dans les vidéos** :  
   L’homographie est utilisée pour le suivi d'objets dans des séquences vidéo, en ajustant les coordonnées des objets dans différentes images pour suivre leurs déplacements dans une scène 2D.

# Indices de confiance pour évaluer la fiabilité du résultat

Une fois l’homographie estimée, il est important de mesurer la qualité de cette estimation. Voici quelques indices de confiance utilisés pour évaluer la fiabilité de l’homographie :

1. **Erreur de projection** :  
   L'un des indices les plus simples consiste à mesurer l'erreur de projection des points transformés par l'homographie. Cela consiste à appliquer l’homographie estimée aux points de l'image d’origine et comparer les points projetés aux points correspondants dans l’image cible. Une petite erreur de projection (ou résidu) indique que l’homographie est fiable.

   L'erreur de projection est calculée par :
   $$ [
   \text{Erreur} = \left\| \mathbf{p}' - H \mathbf{p} \right\|
   ] $$
   où \( \mathbf{p} \) est un point dans l'image source et \( \mathbf{p}' \) est son point correspondant dans l'image cible.

2. **Score RANSAC (Random Sample Consensus)** :  
   RANSAC est une méthode robuste utilisée pour estimer des modèles tout en rejetant les correspondances incorrectes (outliers). Dans le contexte de l’homographie, RANSAC permet de calculer l’homographie tout en écartant les points qui ne correspondent pas bien à la transformation projetée. Le score RANSAC est un bon indice de confiance : un score élevé indique que la majorité des points de correspondance sont corrects, tandis qu’un score faible suggère que l’homographie n'est pas fiable.

3. **Inliers et outliers** :  
   Les points **inliers** sont ceux qui respectent bien l’homographie estimée, tandis que les **outliers** sont des points qui ne correspondent pas à la transformation. Une grande proportion d’inliers par rapport aux outliers signifie que l’homographie est robuste et fiable. Si la proportion d'inliers est faible, cela indique une estimation erronée.

4. **Matrice de covariance des paramètres de l’homographie** :  
   Une autre méthode pour évaluer la fiabilité de l’homographie consiste à analyser la **matrice de covariance** des paramètres estimés. Cette matrice donne une mesure de l'incertitude associée aux coefficients de l’homographie. Si la covariance est faible, cela signifie que les paramètres sont estimés avec une bonne précision. Si elle est élevée, cela indique une incertitude élevée dans l’estimation des paramètres de l’homographie.

5. **Validation visuelle** :  
   Une méthode simple mais efficace est de visualiser l’homographie estimée sur un échantillon de points. Si l’homographie transforme les points d'une image de manière cohérente avec ceux de l’autre image, l'estimation est probablement fiable. L'utilisation de logiciels comme OpenCV permet de vérifier visuellement l'exactitude de l'homographie estimée.


Q4 : Modifier le script fourni afin de saisir les points par paires entre deux images, de façon à réaliser un panorama comme dans l'exemple de la Figure 2.

Voici le code pour créer un panorama :

In [ ]:
import numpy as np
import cv2
import os

# Fonction pour sélectionner des points dans deux images
def select_points(event, x, y, flags, param):
    global points_selected, img1_points, img2_points, img1, img2, img1_clone, img2_clone, current_image
    if event == cv2.EVENT_LBUTTONDOWN:
        points_selected += 1
        if current_image == 1:
            cv2.circle(img1, (x, y), 5, (0, 255, 0), -1)
            img1_points.append([x, y])
        elif current_image == 2:
            cv2.circle(img2, (x, y), 5, (255, 0, 0), -1)
            img2_points.append([x, y])
    elif event == cv2.EVENT_RBUTTONDOWN:  # Réinitialiser
        points_selected = 0
        img1_points = []
        img2_points = []
        img1 = img1_clone.copy()
        img2 = img2_clone.copy()

# Charger les images
img1_path = "./Images_Homographie/keble_a.jpg"
img2_path = "./Images_Homographie/keble_b.jpg"

if not os.path.exists(img1_path) or not os.path.exists(img2_path):
    print("Erreur : Une ou les deux images sont introuvables.")
    exit(1)

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

if img1 is None or img2 is None:
    print("Erreur : Impossible de charger une ou les deux images.")
    exit(1)

# Cloner les images pour les réinitialisations
img1_clone = img1.copy()
img2_clone = img2.copy()

# Variables pour stocker les points
points_selected = 0
img1_points = []
img2_points = []
current_image = 1

# Configuration de la fenêtre pour sélectionner les points
cv2.namedWindow("Image 1")
cv2.namedWindow("Image 2")
cv2.setMouseCallback("Image 1", select_points)
cv2.setMouseCallback("Image 2", select_points)

print("Cliquez pour sélectionner des points correspondants dans les deux images.")
print("Cliquez sur 'q' pour passer à l'image suivante et terminer la sélection.")

# Sélectionner les points dans les deux images
while True:
    cv2.imshow("Image 1", img1)
    cv2.imshow("Image 2", img2)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q") and points_selected >= 4:
        if current_image == 1:
            current_image = 2
        else:
            break

cv2.destroyAllWindows()

# Vérification des points sélectionnés
if len(img1_points) < 4 or len(img2_points) < 4:
    print("Erreur : Vous devez sélectionner au moins 4 points dans chaque image.")
    exit(1)

# Conversion en numpy arrays
img1_points = np.array(img1_points, dtype=np.float32)
img2_points = np.array(img2_points, dtype=np.float32)

# Calculer l'homographie entre les deux images
H, status = cv2.findHomography(img2_points, img1_points, method=cv2.RANSAC)
print(f"Matrice H estimée :\n{H}")

# Appliquer l'homographie pour transformer la deuxième image
height, width, _ = img1.shape
img2_transformed = cv2.warpPerspective(img2, H, (width * 2, height))

# Fusionner les deux images pour créer le panorama
panorama = img2_transformed.copy()
panorama[0:img1.shape[0], 0:img1.shape[1]] = img1

# Afficher et sauvegarder le panorama
cv2.imshow("Panorama", panorama)
key = cv2.waitKey(0)
if key == ord("s"):
    cv2.imwrite("panorama.png", panorama)
    print("Panorama sauvegardé sous 'panorama.png'")
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/youssef/Desktop/Labs_Computer_Vision/new-venv/lib/python3.12/site-packages/cv2/qt/plugins"


Cliquez pour sélectionner des points correspondants dans les deux images.
Cliquez sur 'q' pour passer à l'image suivante et terminer la sélection.


# Conditions pour la prise de vue

Pour que cette méthode fonctionne bien, la prise de vue entre les deux images doit respecter les conditions suivantes :

- **Les images doivent partager un champ de vision similaire.** Cela signifie que les images doivent être prises avec une certaine superposition entre elles. Une bonne règle de base est de s'assurer que chaque image couvre environ 30 à 50% de la zone de l'image précédente.
  
- **Les perspectives doivent être raisonnablement proches.** Les images ne doivent pas être prises avec des rotations extrêmes ou des angles de vue trop différents. Sinon, l’homographie peut ne pas être estimée correctement.

- **Il doit y avoir suffisamment de points correspondants.** Plus vous avez de points correspondants bien répartis entre les deux images, plus l'estimation de l'homographie sera précise et robuste.

### Retrouver les paramètres de la transformation à partir de l’homographie

Les paramètres de la transformation projetée entre les deux images sont contenus dans la matrice d’homographie \( H \) estimée. Cette matrice est une transformation projective qui peut être utilisée pour calculer la transformation d’un plan sur un autre. La matrice \( H \) contient les coefficients qui définissent la relation entre les coordonnées des points dans l'image source et ceux dans l'image cible :

$$ [
H = \begin{pmatrix} 
h_{11} & h_{12} & h_{13} \\
h_{21} & h_{22} & h_{23} \\
h_{31} & h_{32} & h_{33}
\end{pmatrix}
]$$

- Les coefficients  \( h_{11}, h_{12}, h_{21}, h_{22}, h_{13}, h_{23} \) définissent les transformations affines et projectives.
- \( h_{31}, h_{32} \) affectent la perspective et la déformation.
- \( h_{33} \) est souvent normalisé à 1 pour simplifier les calculs.

Cette matrice permet de transformer un point \( (x, y) \) de l’image source en un point \( (x', y') \) de l’image cible à l'aide de la relation suivante :
\[
\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} = H \cdot \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
\]

La matrice \( H \) permet donc de retrouver les transformations géométriques entre les deux images (translation, rotation, et déformation de perspective).